In [7]:
import gmaps
from config2 import g_key
import pandas as pd 
import requests

In [13]:

# 1. Import the WeatherPy_database.csv file. 
city_data_df= pd.read_csv('weather_data/WeatherPy_Database.csv')

# 1. Import the WeatherPy_database.csv file. 
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description,Date
0,0,Caravelas,-17.71,-39.25,71.47,93,100,13.31,BR,moderate rain,2020-11-01 13:52:38
1,1,Goroka,-6.08,145.39,60.94,87,77,0.92,PG,light rain,2020-11-01 13:52:38
2,2,Yar-Sale,66.83,70.83,32.97,96,100,18.23,RU,overcast clouds,2020-11-01 13:52:39
3,3,Ushuaia,-54.80,-68.30,57.34,64,49,7.67,AR,scattered clouds,2020-11-01 13:52:39
4,4,Vao,-22.67,167.48,73.98,85,21,18.92,NC,few clouds,2020-11-01 13:52:39


In [14]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("Enter minimum temperature:"))
max_temp = float(input("Enter maximum temperature:"))

Enter minimum temperature:70
Enter maximum temperature:90


In [16]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] > min_temp)]
clean_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description,Date
0,0,Caravelas,-17.71,-39.25,71.47,93,100,13.31,BR,moderate rain,2020-11-01 13:52:38
4,4,Vao,-22.67,167.48,73.98,85,21,18.92,NC,few clouds,2020-11-01 13:52:39
6,6,Rikitea,-23.12,-134.97,74.70,81,36,9.15,PF,scattered clouds,2020-11-01 13:52:39
8,8,Harper,4.38,-7.72,82.18,76,47,8.46,LR,light rain,2020-11-01 13:52:39
12,12,Cape Town,-33.93,18.42,78.80,41,0,16.11,ZA,clear sky,2020-11-01 13:52:40
...,...,...,...,...,...,...,...,...,...,...,...
683,683,Maningrida,-12.07,134.27,83.03,71,23,5.95,AU,few clouds,2020-11-01 13:53:49
687,687,Kabwe,-14.45,28.45,86.09,26,35,8.16,ZM,scattered clouds,2020-11-01 13:53:50
690,690,Manaus,-3.10,-60.02,77.00,83,75,6.93,BR,broken clouds,2020-11-01 13:50:17
691,691,Barahona,18.21,-71.10,78.80,88,20,10.36,DO,few clouds,2020-11-01 13:53:50


In [18]:
# 4a. Determine if there are any empty rows.
clean_df.isnull().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                2
Current Description    0
Date                   0
dtype: int64

In [19]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
new_df = clean_df.dropna()
new_df.count()


City_ID                292
City                   292
Lat                    292
Lng                    292
Max Temp               292
Humidity               292
Cloudiness             292
Wind Speed             292
Country                292
Current Description    292
Date                   292
dtype: int64

In [20]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Caravelas,BR,71.47,moderate rain,-17.71,-39.25,
4,Vao,NC,73.98,few clouds,-22.67,167.48,
6,Rikitea,PF,74.70,scattered clouds,-23.12,-134.97,
8,Harper,LR,82.18,light rain,4.38,-7.72,
12,Cape Town,ZA,78.80,clear sky,-33.93,18.42,
13,Maragogi,BR,83.28,overcast clouds,-9.01,-35.22,
15,Saint-Philippe,RE,75.20,few clouds,-21.36,55.77,
16,Sao Filipe,CV,81.19,clear sky,14.90,-24.50,
21,Livingstonia,MW,73.54,light rain,-10.61,34.11,
24,Hermanus,ZA,78.01,clear sky,-34.42,19.23,


In [21]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index,row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotel_data = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_name = hotel_data['results'][0]['name']
        hotel_df.loc[index,'Hotel Name'] = hotel_name
    except IndexError:
        print('Hotel name not found for city: ',row['City'],'-Skipping')
        pass

Hotel name not found for city:  Ormara -Skipping
Hotel name not found for city:  Boende -Skipping
Hotel name not found for city:  Sao Paulo De Olivenca -Skipping
Hotel name not found for city:  Pouebo -Skipping
Hotel name not found for city:  Barra Patuca -Skipping
Hotel name not found for city:  Mitu -Skipping
Hotel name not found for city:  Tiznit -Skipping
Hotel name not found for city:  Caconda -Skipping
Hotel name not found for city:  Kieta -Skipping
Hotel name not found for city:  Kibala -Skipping
Hotel name not found for city:  Kalakamati -Skipping
Hotel name not found for city:  Manaquiri -Skipping
Hotel name not found for city:  Gamba -Skipping
Hotel name not found for city:  Morros -Skipping
Hotel name not found for city:  San Borja -Skipping
Hotel name not found for city:  Nalut -Skipping
Hotel name not found for city:  Obo -Skipping
Hotel name not found for city:  Tall Kayf -Skipping
Hotel name not found for city:  Mweka -Skipping
Hotel name not found for city:  Adrar -Skip

In [23]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Caravelas,BR,71.47,moderate rain,-17.71,-39.25,Pousada dos Navegantes
4,Vao,NC,73.98,few clouds,-22.67,167.48,Hotel Kou-Bugny
6,Rikitea,PF,74.70,scattered clouds,-23.12,-134.97,Pension Maro'i
8,Harper,LR,82.18,light rain,4.38,-7.72,Screensaver
12,Cape Town,ZA,78.80,clear sky,-33.93,18.42,Southern Sun Waterfront Cape Town


In [24]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [25]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [26]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))